In [1]:
import requests
import re
import json
import pandas as pd
from datetime import datetime, timedelta
import ast
import warnings
warnings.simplefilter("ignore", UserWarning)

In [ ]:

URL_BASE = "https://www.google.com/finance/_/GoogleFinanceUi/data/batchexecute"

ativo = 'TAEE11'

params = {
    "rpcids": "AiCwsd",
    "source-path": "/finance/quote/BBAS3:BVMF",
    "f.sid": "-1601762218538142130",
    "bl": "boq_finance-ui_20251010.01_p1",
    "hl": "pt",
    "soc-app": "162",
    "soc-platform": "1",
    "soc-device": "1",
    "_reqid": "39577530",
    "rt": "c",
}

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36",
    "Content-Type": "application/x-www-form-urlencoded;charset=utf-8",
    "Referer": "https://www.google.com/finance/quote/BBAS3:BVMF"
}

payload = (
    f'f.req=[[["AiCwsd","[[[null,[\\"{ativo}\\",\\"BVMF\\"]]],1,null,null,null,null,null,0]",null,"generic"]]]'
    '&at=ANXjBT-5oBApzzVabFGFEcU480EA:1760488320351'
)


In [5]:
def get_api_data(ativo):
    payload = (
    f'f.req=[[["AiCwsd","[[[null,[\\"{ativo}\\",\\"BVMF\\"]]],1,null,null,null,null,null,0]",null,"generic"]]]'
    '&at=ANXjBT-5oBApzzVabFGFEcU480EA:1760488320351'
    )

    response = requests.post(URL_BASE, params=params, headers=headers, data=payload)
    
    if response.status_code != 200:
        print(f"Erro na coleta de dados do ativo {ativo}!")
        text = []
    
    else:
        text = response.text
    
    return text



In [6]:
def bronze_cleanup(text, ativo):
    
    pattern = r'\[\[(\d{4}),(\d{1,2}),(\d{1,2}),(\d{1,2}),(\d{1,2}),null,null,\[(-?\d+)\]\],\[(-?\d+(?:\.\d+)?),(-?\d+(?:\.\d+)?),(-?\d+(?:\.\d+)?),(\d+),(\d+),(\d+)\],(\d+)\]'

    finds = re.findall(pattern, text, re.DOTALL)

    bronze = pd.DataFrame(finds)
    
    bronze['price'] = bronze[6]
    bronze['timestamp_str'] = bronze[0] + '-' + bronze[1] + '-' + bronze[2] + ' ' + bronze[3] + ':' + bronze[4]
    bronze['timestamp'] = pd.to_datetime(bronze['timestamp_str'])
    bronze['ativo'] = ativo

    silver = bronze[['ativo', 'timestamp', 'price']]
    
    return silver

In [7]:
lista_de_ativos = ['TAEE11', 'PETR4', 'ABEV3']

gold = pd.DataFrame(columns=['ativo', 'timestamp', 'price'])

In [8]:
for ativo in lista_de_ativos:
    
    text = get_api_data(ativo)
    
    silver = bronze_cleanup(text, ativo)
    
    print(silver)
    
    gold = pd.concat([gold, silver])
    
gold = gold[['ativo', 'timestamp', 'price']]

gold

      ativo           timestamp  price
0    TAEE11 2025-11-03 10:01:00   38.1
1    TAEE11 2025-11-03 10:02:00  38.06
2    TAEE11 2025-11-03 10:03:00  38.15
3    TAEE11 2025-11-03 10:05:00  38.03
4    TAEE11 2025-11-03 10:06:00  38.03
..      ...                 ...    ...
392  TAEE11 2025-11-03 16:52:00  38.47
393  TAEE11 2025-11-03 16:53:00  38.47
394  TAEE11 2025-11-03 16:54:00  38.46
395  TAEE11 2025-11-03 16:55:00  38.48
396  TAEE11 2025-11-03 16:56:00  38.47

[397 rows x 3 columns]


C:\Users\Maicon Ferreira\AppData\Local\Temp\ipykernel_4660\4158791434.py:9: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  gold = pd.concat([gold, silver])


     ativo           timestamp  price
0    PETR4 2025-11-03 10:04:00  29.87
1    PETR4 2025-11-03 10:05:00  29.91
2    PETR4 2025-11-03 10:06:00  29.94
3    PETR4 2025-11-03 10:07:00   29.9
4    PETR4 2025-11-03 10:08:00  29.88
..     ...                 ...    ...
402  PETR4 2025-11-03 16:52:00  30.04
403  PETR4 2025-11-03 16:53:00  30.04
404  PETR4 2025-11-03 16:54:00  30.04
405  PETR4 2025-11-03 16:55:00  30.03
406  PETR4 2025-11-03 16:56:00  30.04

[407 rows x 3 columns]
     ativo           timestamp  price
0    ABEV3 2025-11-03 10:04:00   12.8
1    ABEV3 2025-11-03 10:05:00   12.8
2    ABEV3 2025-11-03 10:06:00  12.81
3    ABEV3 2025-11-03 10:07:00  12.82
4    ABEV3 2025-11-03 10:08:00  12.83
..     ...                 ...    ...
402  ABEV3 2025-11-03 16:52:00  12.88
403  ABEV3 2025-11-03 16:53:00  12.89
404  ABEV3 2025-11-03 16:54:00  12.89
405  ABEV3 2025-11-03 16:55:00  12.89
406  ABEV3 2025-11-03 16:56:00  12.88

[407 rows x 3 columns]


,ativo,timestamp,price
0,TAEE11,2025-11-03 10:01:00,38.1
1,TAEE11,2025-11-03 10:02:00,38.06
2,TAEE11,2025-11-03 10:03:00,38.15
3,TAEE11,2025-11-03 10:05:00,38.03
4,TAEE11,2025-11-03 10:06:00,38.03
...,...,...,...
402,ABEV3,2025-11-03 16:52:00,12.88
403,ABEV3,2025-11-03 16:53:00,12.89
404,ABEV3,2025-11-03 16:54:00,12.89
405,ABEV3,2025-11-03 16:55:00,12.89
